In [1]:
try:
    import GetOldTweets3 as got
except:
    !pip install GetOldTweets3
    import GetOldTweets3 as got

In [2]:
try:
    from bs4 import BeautifulSoup
except:
    !pip install beautifulsoup4
    from bs4 import BeautifulSoup

In [3]:
from random import uniform
from tqdm import tqdm_notebook
import pandas as pd
import re
import string

In [6]:
import datetime

days_range = []

#수집 기간 설정
start = datetime.datetime.strptime("2024-09-25", "%Y-%m-%d")
end = datetime.datetime.strptime("2024-10-26", "%Y-%m-%d")
date_generated = [start + datetime.timedelta(days=x) for x in range(0, (end-start).days)]

for date in date_generated:
    days_range.append(date.strftime("%Y-%m-%d"))
    
print("=== 설정된 트윗 수집 기간은 {} 에서 {}까지 입니다. ===".format(days_range[0], days_range[-1]))
print("=== 총 {}일 간의 데이터 수집 ===".format(len(days_range)))

=== 설정된 트윗 수집 기간은 2024-09-25 에서 2024-10-25까지 입니다. ===
=== 총 31일 간의 데이터 수집 ===


In [11]:
# 방법 1: snscrape 사용
try:
    import snscrape.modules.twitter as sntwitter
except:
    !pip install snscrape
    import snscrape.modules.twitter as sntwitter

def collect_tweets_snscrape(search_query, start_date, end_date, limit=1000):
    tweets = []
    query = f"{search_query} since:{start_date} until:{end_date}"
    
    for tweet in sntwitter.TwitterSearchScraper(query).get_items():
        if len(tweets) >= limit:
            break
        tweets.append({
            'date': tweet.date,
            'content': tweet.rawContent,
            'username': tweet.user.username,
            'likes': tweet.likeCount
        })
    
    return pd.DataFrame(tweets)

# 방법 2: Selenium을 사용한 웹 스크래핑
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

def collect_tweets_selenium(search_query, scroll_count=5):
    tweets = []
    driver = webdriver.Chrome()  # ChromeDriver 필요
    
    try:
        driver.get(f"https://twitter.com/search?q={search_query}&src=typed_query")
        time.sleep(3)  # 페이지 로딩 대기
        
        for _ in range(scroll_count):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            
        tweet_elements = driver.find_elements(By.CSS_SELECTOR, '[data-testid="tweet"]')
        for element in tweet_elements:
            tweet_text = element.find_element(By.CSS_SELECTOR, '[data-testid="tweetText"]').text
            tweets.append(tweet_text)
            
    finally:
        driver.quit()
    
    return pd.DataFrame(tweets, columns=['content'])

# 사용 예시
search_word = '흑백요리사'
start_date = '2023-09-25'
end_date = '2023-10-26'


# selenium 사용
df_selenium = collect_tweets_selenium(search_word)
df_selenium.to_excel(f"tweet_list_selenium_{search_word}.xlsx")

ModuleNotFoundError: No module named 'openpyxl'

In [ ]:
# snscrape 사용
df_snscrape = collect_tweets_snscrape(search_word, start_date, end_date)
df_snscrape.to_excel(f"tweet_list_snscrape_{search_word}.xlsx")


In [9]:
!pip install selenium

  Using cached websocket_client-1.8.0-py3-none-any.whl.metadata (8.0 kB)
  Using cached cffi-1.17.1-cp310-cp310-win_amd64.whl.metadata (1.6 kB)
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   -------------------- ------------------- 5.0/9.7 MB 25.1 MB/s eta 0:00:01
   ------------------------------------- -- 9.2/9.7 MB 22.0 MB/s eta 0:00:01
   ---------------------------------------- 9.7/9.7 MB 20.1 MB/s eta 0:00:00
Using cached websocket_client-1.8.0-py3-none-any.whl (58 kB)
Using cached cffi-1.17.1-cp310-cp310-win_amd64.whl (181 kB)
Using cached pycparser-2.22-py3-none-any.whl (117 kB)


In [7]:
import time

start_date = days_range[0]
end_date = (datetime.datetime.strptime(days_range[-1], "%Y-%m-%d") 
            + datetime.timedelta(days=1)).strftime("%Y-%m-%d")

# 검색어 = #슬픔, #극혐, #무섭, 기쁨 
# '극혐', '슬픔', '공포', '기쁨' 시신 or 살인 or 납치 or 유괴 or 변사체
# 공포 정서에 공포영화를 넣어보는 것도 좋을 듯
# 공포 - 뱀, 바퀴벌레, 바선생, 사이비, 공포증, 뱀, 거미, 악천후, 벌레
# 5일치 - ㄱㅇㄷ:828, 무지개다리: 102, 딥빡: 367
# 10일치 - 개빡: 1953
#무지개다리 별로 -> 너무 적음, 유기견, 안락사로 대체
search_word_list = ['흑백요리사']
for search_word in search_word_list:
    print(search_word)
    tweetCriteria = got.manager.TweetCriteria().setQuerySearch(search_word).setSince(start_date).setUntil(end_date).setMaxTweets(-1)
    print("Collecting data start.. from {} to {}".format(days_range[0], days_range[-1]))
    start_time = time.time()

    tweet = got.manager.TweetManager.getTweets(tweetCriteria)

    print("Collecting data end.. {0:0.2f} Minutes".format((time.time() - start_time)/60))
    print("=== Total num of tweets is {} ===".format(len(tweet)))
    tweet_list = []


    r = re.compile('#\S+|^https?\S+|@\S+')
    hangul = re.compile('[^ ㄱ-ㅣ가-힣|0-9|!$%&’()*+,.;<=>?\^_`{|}~-]+')
    for index in tqdm_notebook(tweet):
        content = index.text
        content = hangul.sub('',content)
        content = re.sub(r, '', content)
        content = re.sub('  +','',content)
        if content != '' and content != ' ':
            tweet_list.append(content)
    
    twitter_df = pd.DataFrame(tweet_list, columns = ['raw_text'])

    twitter_df.to_excel("tweet_list_{}.xlsx".format(search_word))
    print("=== {} tweets are successfully saved ===".format(len(tweet_list)))


흑백요리사
An error occured during an HTTP request: HTTP Error 403: Forbidden
Try to open in browser: https://twitter.com/search?q=%ED%9D%91%EB%B0%B1%EC%9A%94%EB%A6%AC%EC%82%AC%20since%3A2024-09-25%20until%3A2024-10-26&src=typd


SystemExit: 

c:\Users\joon\AppData\Local\miniconda3\envs\mlops\lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [4]:
df_tweet = pd.read_excel("tweet_list.xlsx")
df_tweet.head(10)

NameError: name 'pd' is not defined